In [1]:
from utils import get_eval_result_from_txt_file, extract_sql_from_json_predictions, convert_text_to_list, result_compare, analysis_compare

# 完整流程展示

## 设置文件路径、tag和数据集

In [2]:
filepath = "/home/jxqi/text2sql/picard/experiment/eval_0504_sparc_add_coref_2432 copy/predictions_eval_None.json"
baseline_filepath = "/home/jxqi/text2sql/picard_train_no_docker/experiment/t5-sparc-0312/predictions_eval_270.2149003147954.json"
dataset = "sparc"
tag_baseline = "baseline"
tag_our_method = "our_method"

## 从json格式的预测结果生成txt格式的结果供下一步进行评估

In [3]:
_, baseline_output_filepath = extract_sql_from_json_predictions(
    baseline_filepath, 
    dataset=dataset, 
    output_dir="default",
    tag = tag_baseline
)
_, output_filepath = extract_sql_from_json_predictions(
    filepath, 
    dataset=dataset, 
    output_dir="default",
    tag = tag_our_method
)

filepath:  ./output/output_sparc_baseline_20220505_104339.txt


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1624/1624 [00:00<00:00, 831082.20it/s]


filepath:  ./output/output_sparc_our_method_20220505_104339.txt


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1624/1624 [00:00<00:00, 544531.91it/s]


## 传入两个模型的预测输出txt，得到预测信息

In [4]:
predicted_file_path = baseline_output_filepath
result_baseline = get_eval_result_from_txt_file(
    predicted_file_path, 
    dataset=dataset,
    etype="match",
    test_suit_path="/home/jxqi/text2sql/error_analysis/test-suite-sql-eval/"
)

Evaluating 1th prediction
Right
easy pred: select * from airlines
easy gold: SELECT * FROM AIRLINES

Right
easy pred: select * from airlines where airline = 'JetBlue Airways'
easy gold: SELECT * FROM AIRLINES WHERE Airline  =  "JetBlue Airways"

Right
easy pred: select country from airlines where airline = 'Jetblue Airways'
easy gold: SELECT Country FROM AIRLINES WHERE Airline  =  "JetBlue Airways"

Evaluating 2th prediction
Right
easy pred: select abbreviation from airlines
easy gold: SELECT Abbreviation FROM AIRLINES

Right
easy pred: select abbreviation from airlines where airline = "JetBlue Airways"
easy gold: SELECT Abbreviation FROM AIRLINES WHERE Airline  =  "JetBlue Airways"

Evaluating 3th prediction
Right
medium pred: select airline, abbreviation from airlines
medium gold: SELECT Airline ,  Abbreviation FROM AIRLINES

Right
medium pred: select airline, abbreviation from airlines where country = 'USA'
medium gold: SELECT Airline ,  Abbreviation FROM AIRLINES WHERE Country  =  

In [5]:
predicted_file_path = output_filepath
result_our_method = get_eval_result_from_txt_file(
    predicted_file_path, 
    dataset=dataset,
    etype="match",
    test_suit_path="/home/jxqi/text2sql/error_analysis/test-suite-sql-eval/"
)

Evaluating 1th prediction
Right
easy pred: select * from airlines
easy gold: SELECT * FROM AIRLINES

Right
easy pred: select * from airlines where airline = "Jetblue Airways"
easy gold: SELECT * FROM AIRLINES WHERE Airline  =  "JetBlue Airways"

Right
easy pred: select country from airlines where airline = "Jetblue Airways"
easy gold: SELECT Country FROM AIRLINES WHERE Airline  =  "JetBlue Airways"

Evaluating 2th prediction
Right
easy pred: select abbreviation from airlines
easy gold: SELECT Abbreviation FROM AIRLINES

Right
easy pred: select abbreviation from airlines where airline = "Jetblue Airways"
easy gold: SELECT Abbreviation FROM AIRLINES WHERE Airline  =  "JetBlue Airways"

Evaluating 3th prediction
Right
medium pred: select airline, abbreviation from airlines
medium gold: SELECT Airline ,  Abbreviation FROM AIRLINES

Right
medium pred: select airline, abbreviation from airlines where country = "USA"
medium gold: SELECT Airline ,  Abbreviation FROM AIRLINES WHERE Country  =  

## 然后进行比较和分析

In [6]:
result_baseline_l = result_baseline.split("\n")
result_our_method_l = result_our_method.split("\n")

In [7]:
result_dict = result_compare(baseline_text_l=result_baseline_l, text_l=result_our_method_l)

In [8]:
for k,v in result_dict.items():
    print(k, len(v))

both_right_l 157
both_wrong_l 207
baseline_right_l 16
right_l 42


In [9]:
analysis_compare(result_dict)

Interaction context:
Goal:  What is the airport code of the airport with the most flights?
------------------------------------------------------------------------------------------------------------------------------------------------
Question # 0 :  What are airport codes of any airport with a arriving flight?
Baseline predition:    select t1.airportcode from airports as t1 join flights as t2 on t1.airportcode = t2.sourceairport
Our method predition:  select airportcode from flights as t1 join airports as t2 on t1.destairport = t2.airportcode

Question # 1 :  Also include codes of airports with departure flights.
^^^^^^^^^^^^^^^^^^^^^^^
^^^^^Please note: ^^^^^
Baseline predition:    select t1.airportcode from airports as t1 join flights as t2 on t1.airportcode = t2.sourceairport where t2.flightno = 1 or t2.flightno = 2
Our method predition:  select airportcode from airports as t1 join flights as t2 on t1.airportcode = t2.sourceairport or t1.airportcode = t2.destairport
^^^^^^^^^^^^^^

# 测试SParC的2112

In [ ]:
filepath = "/home/jxqi/text2sql/picard_train_no_docker/experiment/t5-sparc-0312/predictions_eval_234.64470094438616.json"
extract_sql_from_json_predictions(
    filepath, 
    dataset="sparc", 
    output_dir="default",
    tag = "ckpt_2112_sparc_no_picard"
)

In [ ]:
predicted_file_path = "./output/output_sparc_ckpt_2112_sparc_no_picard_20220430_222747.txt"
get_eval_result_from_txt_file(
    predicted_file_path, 
    dataset="sparc",
    etype="match"
)

# 测试SParC的2432

In [ ]:
filepath = "/home/jxqi/text2sql/picard_train_no_docker/experiment/t5-sparc-0312/predictions_eval_270.2149003147954.json"
extract_sql_from_json_predictions(
    filepath, 
    dataset="sparc", 
    output_dir="default",
    tag = "ckpt_2112_sparc_no_picard"
)

In [ ]:
predicted_file_path = "./output/output_sparc_ckpt_2112_sparc_no_picard_20220430_223322.txt"
get_eval_result_from_txt_file(
    predicted_file_path, 
    dataset="sparc",
    etype="exec"
)

In [ ]:
filepath = "/home/jxqi/text2sql/picard/experiment/t5-cosql-0408-add-coref/predictions_eval_159.99272903538537.json"
extract_sql_from_json_predictions(
    filepath, 
    dataset="cosql", 
    output_dir="default",
    tag = "ckpt_1280_cosql_no_picard"
)

In [ ]:
predicted_file_path = "./output/output_cosql_ckpt_1280_cosql_no_picard_20220430_232746.txt"
get_eval_result_from_txt_file(
    predicted_file_path, 
    dataset="cosql",
    etype="match"
)

In [ ]:
predicted_file_path = "./output/output_cosql_ckpt_1280_cosql_no_picard_20220430_232746.txt"
get_eval_result_from_txt_file(
    predicted_file_path, 
    dataset="cosql",
    etype="exec"
)

# 测试

In [ ]:
predicted_file_path = "/home/jxqi/Text-to-SQL/t5/experiment/t5-large_9_26_1_6/epoch_2_predictions.txt"
cmd = 'python /home/jxqi/Text-to-SQL/t5/evaluate/test-suite-sql-eval-master/evaluation.py --gold /home/jxqi/Text-to-SQL/t5/dataset/spider/dev_gold.sql --pred {0} --etype match --db /home/jxqi/Text-to-SQL/t5/dataset/spider/database/ --table /home/jxqi/Text-to-SQL/t5/dataset/spider/tables.json'.format(predicted_file_path)
result = os.popen(cmd).read()

In [ ]:
get_eval_result_from_txt_file("./output/output_spider_ckpt_2624_beam_32_ex_8036_20220422_161808.txt", etype="match")

In [ ]:
get_eval_result_from_txt_file("./output/output_spider_picard_output.txt", etype="match")

In [ ]:
get_eval_result_from_txt_file("./output/output_spider_picard2.txt", etype="match")

In [ ]:
get_eval_result_from_txt_file("./output/output_spider_picard_output.txt", etype="exec")

In [ ]:
get_eval_result_from_txt_file("./output/output_spider_ckpt_2624_beam_32_ex_8036_20220422_161808.txt", etype="exec")

In [ ]:
get_eval_result_from_txt_file("./output/output_spider_picard2.txt", etype="exec")

# 测试SParC 2432 add_coref

In [ ]:
filepath = "/home/jxqi/text2sql/picard/experiment/eval_0504_sparc_add_coref_2432 copy/predictions_eval_None.json"
extract_sql_from_json_predictions(
    filepath, 
    dataset="sparc", 
    output_dir="default",
    tag = "ckpt_2432_sparc_use_picard"
)

In [ ]:
predicted_file_path = "./output/output_sparc_ckpt_2432_sparc_use_picard_20220504_152431.txt"
get_eval_result_from_txt_file(
    predicted_file_path, 
    dataset="sparc",
    etype="match",
    test_suit_path="/home/jxqi/text2sql/error_analysis/test-suite-sql-eval/"
)

In [ ]:
predicted_file_path = "./output/output_sparc_ckpt_2432_sparc_use_picard_20220504_114246.txt"
get_eval_result_from_txt_file(
    predicted_file_path, 
    dataset="sparc",
    etype="exec"
)

# 读取dev.json做上下文对话的分析

In [ ]:
import json

In [ ]:
dev = json.load(open("/home/jxqi/text2sql/error_analysis/data/sparc/dev.json", "r"))

In [ ]:
def get_all_context_from_dev(dev_filepath="/home/jxqi/text2sql/error_analysis/data/sparc/dev.json"):
    '''
    Args:
        dev_filepath: dev.json文件的路径
        
    Returns:
        dev_context_l: 存放每一个interaction的context的列表，其中每个元素为一个字典，包括"goal"为交互目标，"interactions"为对话上下文
    '''
    dev = json.load(open(dev_filepath, "r"))
    dev_context_l = []
    for item in dev:
        final = item["final"]["utterance"]
        interactions = [i["utterance"] for i in item["interaction"]] 
        context = {
            "goal": goal,
            "interactions": interactions
        }
        dev_context_l.append(context)
        
    return dev_context_l

In [ ]:
dev_context_l = get_all_context_from_dev()

In [ ]:
dev_context_l